In [12]:
import os
import time
from datetime import datetime
from typing import Callable

from memory_profiler import memory_usage
from texttable import Texttable


dir_name = 'test_data/'


class TestCase:
    
    def __init__(self, name: str, in_file: str, out_file: str):
        self.name = name
        self.in_data = self.read_file_plain(in_file)
        self.out_data = self.read_file_plain(out_file)
        
    def read_file_plain(self, filename):
        f = open(filename, "r")
        lines = f.readlines()
        if lines:
            return lines[0].strip()
        else:
            raise ReadFileError(f"File named {filename} is empty!")
    
    def _run_test(self, testing_func: Callable, args, kwargs):
        args = (self.in_data, *args)
        real_result = testing_func(*args, **kwargs)
        test_result = {
            "test_name": self.name,
            "is_passed": "PASSED" if real_result == self.out_data else "FAILED"
        }
        return test_result
    
    def _gather_metrics(self, testing_func: Callable, args, kwargs):
        start = time.time()
        result = self._run_test(testing_func, args, kwargs)
        end = time.time()
        _metric_execution_time = ("{:.6f}".format(end - start))
        result["execution_time"] = _metric_execution_time
        
        inaccurate_mem_usage = memory_usage((testing_func, (self.in_data, *args), kwargs))
        inaccurate_mem_usage = sum(inaccurate_mem_usage)/len(inaccurate_mem_usage)
        _metric_rude_mem_usage = ("{:.6f}".format(inaccurate_mem_usage))
        result["rude_mem_usage"] = _metric_rude_mem_usage
        
        return result
    
    def run(self, testing_func: Callable, func_args, func_kwargs):
        return self._gather_metrics(testing_func, func_args, func_kwargs)


class TestRunner:
    
    def __init__(self, dir_name="test_data/"):
        self.data_dir = dir_name
        # self.validate_data_existance() unimplemented yet; TODO: validate if there are needed dir & it's not empty
        self.test_case_class = TestCase
        self.test_case_collection = self.build_test_case_collection()
        self.full_run_results = None
    
    def build_test_case_collection(self):
        test_collection = []

        tmp_collection = self.gather_tmp_test_data()

        for test_name in sorted(tmp_collection.keys()):
            test_case = self.test_case_class(
                test_name,
                self.data_dir + tmp_collection[test_name]["in"],
                self.data_dir + tmp_collection[test_name]["out"]
            )
            test_collection.append(test_case)

        return test_collection

    def gather_tmp_test_data(self) -> dict:
        is_test_file = lambda x: x if (x.endswith(".in") or x.endswith(".out")) else False
        test_names = filter(lambda x: is_test_file(x), os.listdir(self.data_dir))
        test_names = list(set(map(lambda x: x[:-3] if x.endswith(".in") else x[:-4], test_names)))
        tmp_collection = {test_name: {} for test_name in test_names}

        for filename in os.listdir(self.data_dir):
            if filename.endswith(".in"):
                test_name = filename[:-3]
                tmp_collection[test_name]["in"] = filename
            elif filename.endswith(".out"):
                test_name = filename[:-4]
                tmp_collection[test_name]["out"] = filename

        return tmp_collection
    
    def run_tests(self, func_to_test: Callable, func_args=None, func_kwargs=None):
        func_args = func_args or ()
        func_kwargs = func_kwargs or {}
        test_results = []
        for test_case in self.test_case_collection:
            test_result = test_case.run(func_to_test, func_args, func_kwargs)
            test_results.append(self.format_result(test_result))

        self.full_run_results = test_results
        return test_results
    
    def format_result(self, result):
        result_dict = {
            "result_datetime": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
            "result": result
        }
        return result_dict
    
    def render_results_as_raw_text(self):
        all_results_string = ""
        for result in self.full_run_results:
            all_results_string = all_results_string + self.render_result_as_raw_text(result) + "\n\n"
        return all_results_string
            
    def render_result_as_raw_text(self, result):
        test_res = result.get("result")
        test_name = "Test name: " + test_res["test_name"] + "\n"
        test_status = "Test passed: " + str(test_res["is_passed"]) + "\n"
        exec_time = "Executed in: " + test_res["execution_time"] + " seconds" + "\n"
        rude_memory_usage = "Memory used on execution: " + test_res["rude_mem_usage"] + "\n"
        return test_name + test_status + exec_time + rude_memory_usage
    
    def render_as_ascii_table(self) -> str:
        table = Texttable()
        table.set_cols_align(("c", "c", "c", "c"))
        table.set_cols_valign(("m", "m", "m", "m"))        
        rows_to_add = [["Test Name", "Status", "Executed in (sec)", "Memory used (Mb)"],]
        for res in self.full_run_results:
            test_res = res.get("result")
            res_list = [test_res["test_name"], str(test_res["is_passed"]), test_res["execution_time"], test_res["rude_mem_usage"]]
            rows_to_add.append(res_list)
        table.set_cols_dtype(["t", "t", "f", "f"])
        table.set_precision(6)
        table.add_rows(rows_to_add)
        return table.draw()

runner = TestRunner()

def measure_len(string_to_measure):
    return str(len(string_to_measure))

runner.run_tests(measure_len,)
# print(runner.render_results_as_raw_text())

ascii_table_result = runner.render_as_ascii_table()

print(ascii_table_result)

# TODO:
# - Add time measurement +
# - *Add memory usage measurement ??? +
# - Add multiple run for average results
# - Rendering results into pretty ASCII table in console
# - Put it into separate lib
# - Create simple string-len script
# - Import test_lib into string-len script
# - Use it

+-----------+--------+-------------------+------------------+
| Test Name | Status | Executed in (sec) | Memory used (Mb) |
+===========+========+===================+==================+
|  test.0   | PASSED |     0.000018      |    90.628906     |
+-----------+--------+-------------------+------------------+
|  test.1   | PASSED |     0.000065      |    90.628906     |
+-----------+--------+-------------------+------------------+
|  test.2   | PASSED |     0.000060      |    90.628906     |
+-----------+--------+-------------------+------------------+
|  test.3   | PASSED |     0.000064      |    90.628906     |
+-----------+--------+-------------------+------------------+
|  test.4   | FAILED |     0.000071      |    90.628906     |
+-----------+--------+-------------------+------------------+


In [41]:
from datetime import datetime

datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

'2021-11-06T17:23:57'